In [1]:
import numpy as np
#import sklearn as sk
#import matplotlib.pyplot as plt
import pandas as pd

#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import Perceptron
#from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import VotingClassifier
#from sklearn import svm

In [2]:
#### READING OUR GIVEN DATA INTO PANDAS DATAFRAME ####
x_train = pd.read_csv('train.csv')
y_train = pd.read_csv('trainLabels.csv')
x_test = pd.read_csv('test.csv')
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_train = y_train.ravel()
print('training_x Shape:',x_train.shape,',training_y Shape:',y_train.shape, ',testing_x Shape:',x_test.shape)

#Checking the models
x_all = np.r_[x_train,x_test]
print('x_all shape :',x_all.shape)

training_x Shape: (999, 40) ,training_y Shape: (999,) ,testing_x Shape: (8999, 40)
x_all shape : (9998, 40)


In [3]:
#### USING THE GAUSSIAN MIXTURE MODEL ####
from sklearn.mixture import GaussianMixture
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a mixture of Gaussians with EM
        gmm = GaussianMixture(n_components=n_components,covariance_type=cv_type)
        gmm.fit(x_all)
        bic.append(gmm.aic(x_all))
        if bic[-1] < lowest_bic:
            print(f'This bic: {bic[-1]}')
            print(f'Current lowest bic: {lowest_bic}')
            lowest_bic = bic[-1]
            best_gmm = gmm

best_gmm.fit(x_all)
x_train = best_gmm.predict_proba(x_train)
x_test = best_gmm.predict_proba(x_test)


This bic: 1641320.4854730715
Current lowest bic: inf
This bic: 1584323.0892013817
Current lowest bic: 1641320.4854730715
This bic: 1542270.881243344
Current lowest bic: 1584323.0892013817
This bic: 1523689.187087485
Current lowest bic: 1542270.881243344
This bic: 1508961.0359495592
Current lowest bic: 1523689.187087485
This bic: 1498347.5814400476
Current lowest bic: 1508961.0359495592
This bic: 909309.3112573357
Current lowest bic: 1498347.5814400476
This bic: 906194.4502941675
Current lowest bic: 909309.3112573357
This bic: 904621.167007703
Current lowest bic: 906194.4502941675
This bic: 903306.4368797743
Current lowest bic: 904621.167007703
This bic: 902258.887177366
Current lowest bic: 903306.4368797743
This bic: 894239.3844794282
Current lowest bic: 902258.887177366
This bic: 866620.615364033
Current lowest bic: 894239.3844794282
This bic: 829040.2702080137
Current lowest bic: 866620.615364033


In [4]:
#### TAKING ONLY TWO MODELS FOR KEEPING IT SIMPLE ####
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

param_grid = dict( )
#### GRID SEARCH for BEST TUNING PARAMETERS FOR KNN #####
grid_search_knn = GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='accuracy').fit(x_train,y_train)
print('best estimator KNN:',grid_search_knn.best_estimator_,'Best Score', grid_search_knn.best_estimator_.score(x_train,y_train))
knn_best = grid_search_knn.best_estimator_

#### GRID SEARCH for BEST TUNING PARAMETERS FOR RandomForest #####
grid_search_rf = GridSearchCV(rf, param_grid=dict( ), verbose=3,scoring='accuracy',cv=10).fit(x_train,y_train)
print('best estimator RandomForest:',grid_search_rf.best_estimator_,'Best Score', grid_search_rf.best_estimator_.score(x_train,y_train))
rf_best = grid_search_rf.best_estimator_

best estimator KNN: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform') Best Score 0.995995995995996
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV]  ................................................................
[CV] .................................... , score=1.000, total=   0.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] .................................... , score=1.000, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=1.000, total=   0.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] .................................... , score=0.990, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=1.000, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=1.000, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=0.990, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=1.000, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=0.980, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=1.000, total=   0.2s
best estimator RandomForest: RandomForestClassifier(bootstrap=True, ccp_alpha

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.8s finished


In [5]:
knn_best.fit(x_train,y_train)
print(knn_best.predict(x_test)[0:10])
rf_best.fit(x_train,y_train)
print(rf_best.predict(x_test)[0:10])

#### SCORING THE MODELS ####
print('Score for KNN :',cross_val_score(knn_best,x_train,y_train,cv=10,scoring='accuracy').mean())
print('Score for Random Forest :',cross_val_score(rf_best,x_train,y_train,cv=10,scoring='accuracy').max())

[0 1 0 0 0 0 1 0 0 1]
[0 1 0 0 0 0 1 0 0 1]
Score for KNN : 0.9960000000000001
Score for Random Forest : 1.0


In [6]:
### IN CASE WE WERE USING MORE THAN ONE CLASSIFIERS THEN VOTING CLASSIFIER CAN BE USEFUL ###
#clf = VotingClassifier(
#		estimators=[('knn_best',knn_best),('rf_best',rf_best)],
#		#weights=[871856020222,0.907895269918]
#	)
#clf.fit(x_train,y_train)
#print clf.predict(x_test)[0:10]


In [7]:
##### FRAMING OUR SOLUTION #####
knn_best_pred = pd.DataFrame(knn_best.predict(x_test))
rf_best_pred = pd.DataFrame(rf_best.predict(x_test))
#voting_clf_pred = pd.DataFrame(clf.predict(x_test))

knn_best_pred.index += 1
rf_best_pred.index += 1
#voting_clf_pred.index += 1

#knn_best_pred.to_csv('knn_best_pred.csv')
rf_best_pred.to_csv('Submission.csv')
#voting_clf_pred.to_csv('voting_clf_pred.csv')